Only keep (not ablate) the query activations (query vector, the output of query weights times inputs) of certain positions. Keep all the key activations; the query positions that were kept will automatically attend to relevant key positions by matrix multiplication.

# Setup
(No need to change anything)

## import libs

In [1]:
# Janky code to do different setup when run in a Colab notebook vs VSCode
DEBUG_MODE = False
try:
    import google.colab
    IN_COLAB = True
    print("Running as a Colab notebook")
    %pip install git+https://github.com/neelnanda-io/TransformerLens.git
    # Install another version of node that makes PySvelte work way faster
    !curl -fsSL https://deb.nodesource.com/setup_16.x | sudo -E bash -; sudo apt-get install -y nodejs
    %pip install git+https://github.com/neelnanda-io/PySvelte.git
except:
    IN_COLAB = False
    print("Running as a Jupyter notebook - intended for development only!")
    from IPython import get_ipython

    ipython = get_ipython()
    # Code to automatically update the HookedTransformer code as its edited without restarting the kernel
    ipython.magic("load_ext autoreload")
    ipython.magic("autoreload 2")

Running as a Colab notebook
  Cloning https://github.com/neelnanda-io/TransformerLens.git to /tmp/pip-req-build-ql09n7e1
  Running command git clone --filter=blob:none --quiet https://github.com/neelnanda-io/TransformerLens.git /tmp/pip-req-build-ql09n7e1
  Resolved https://github.com/neelnanda-io/TransformerLens.git to commit 51771ff42e313eb64ffd95841c0923c0c0865efd
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.0/261.0 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 739.7/739.7 kB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 41.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 70.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.9/417.9 MB 2.6 MB/s eta 0:00:00
    

In [2]:
# Plotly needs a different renderer for VSCode/Notebooks vs Colab argh
import plotly.io as pio

if IN_COLAB or not DEBUG_MODE:
    # Thanks to annoying rendering issues, Plotly graphics will either show up in colab OR Vscode depending on the renderer - this is bad for developing demos! Thus creating a debug mode.
    pio.renderers.default = "colab"
else:
    pio.renderers.default = "png"

In [3]:
# Import stuff
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import einops
from fancy_einsum import einsum
import tqdm.notebook as tqdm
import random
from pathlib import Path
import plotly.express as px
from torch.utils.data import DataLoader

from jaxtyping import Float, Int
from typing import List, Union, Optional
from functools import partial
import copy

import itertools
from transformers import AutoModelForCausalLM, AutoConfig, AutoTokenizer
import dataclasses
import datasets
from IPython.display import HTML

In [4]:
# import pysvelte

import transformer_lens
import transformer_lens.utils as utils
from transformer_lens.hook_points import (
    HookedRootModule,
    HookPoint,
)  # Hooking utilities
from transformer_lens import HookedTransformer, HookedTransformerConfig, FactoredMatrix, ActivationCache

We turn automatic differentiation off, to save GPU memory, as this notebook focuses on model inference not model training.

In [5]:
torch.set_grad_enabled(False)

Plotting helper functions:

In [6]:
def imshow(tensor, renderer=None, **kwargs):
    px.imshow(utils.to_numpy(tensor), color_continuous_midpoint=0.0, color_continuous_scale="RdBu", **kwargs).show(renderer)

def line(tensor, renderer=None, **kwargs):
    px.line(y=utils.to_numpy(tensor), **kwargs).show(renderer)

def scatter(x, y, xaxis="", yaxis="", caxis="", renderer=None, **kwargs):
    x = utils.to_numpy(x)
    y = utils.to_numpy(y)
    px.scatter(y=y, x=x, labels={"x":xaxis, "y":yaxis, "color":caxis}, **kwargs).show(renderer)

## Load Model

Decide which model to use (eg. gpt2-small vs -medium)

In [7]:
model = HookedTransformer.from_pretrained(
    "gpt2-medium",
    center_unembed=True,
    center_writing_weights=True,
    fold_ln=True,
    refactor_factored_attn_matrices=True,
)

Loaded pretrained model gpt2-medium into HookedTransformer


## Import functions from repo

In [8]:
!git clone https://github.com/callummcdougall/ARENA_2.0.git

Cloning into 'ARENA_2.0'...
remote: Enumerating objects: 9100, done.
remote: Counting objects: 100% (1812/1812), done.
remote: Compressing objects: 100% (287/287), done.
remote: Total 9100 (delta 1606), reused 1601 (delta 1522), pack-reused 7288
Receiving objects: 100% (9100/9100), 155.60 MiB | 22.98 MiB/s, done.
Resolving deltas: 100% (5501/5501), done.


In [9]:
cd ARENA_2.0/chapter1_transformers/exercises/part3_indirect_object_identification

/content/ARENA_2.0/chapter1_transformers/exercises/part3_indirect_object_identification


In [10]:
import ioi_circuit_extraction as ioi_circuit_extraction

# test prompts

In [11]:
modeltest = HookedTransformer.from_pretrained("gpt2-medium")

Loaded pretrained model gpt2-medium into HookedTransformer


In [15]:
example_prompt = "2 4 8 16 32 64 128"
example_answer = " 256"
utils.test_prompt(example_prompt, example_answer, modeltest, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', '2', ' 4', ' 8', ' 16', ' 32', ' 64', ' 128']
Tokenized answer: [' 256']


Performance on answer token:
Rank: 0        Logit: 16.64 Prob: 46.88% Token: | 256|

Top 0th token. Logit: 16.64 Prob: 46.88% Token: | 256|
Top 1th token. Logit: 14.96 Prob:  8.68% Token: | 512|
Top 2th token. Logit: 14.59 Prob:  6.00% Token: | 128|
Top 3th token. Logit: 14.59 Prob:  5.98% Token: | 192|
Top 4th token. Logit: 14.14 Prob:  3.84% Token: |
|
Top 5th token. Logit: 13.55 Prob:  2.12% Token: | 176|
Top 6th token. Logit: 13.51 Prob:  2.04% Token: | 160|
Top 7th token. Logit: 12.93 Prob:  1.14% Token: | 144|
Top 8th token. Logit: 12.30 Prob:  0.61% Token: | 240|
Top 9th token. Logit: 12.21 Prob:  0.56% Token: | 168|


Ranks of the answer tokens: [(' 256', 0)]

In [13]:
example_prompt = "16 32 64 128 256"
example_answer = " 512"
utils.test_prompt(example_prompt, example_answer, modeltest, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', '16', ' 32', ' 64', ' 128', ' 256']
Tokenized answer: [' 512']


Performance on answer token:
Rank: 1        Logit: 13.77 Prob: 11.90% Token: | 512|

Top 0th token. Logit: 14.20 Prob: 18.24% Token: | 256|
Top 1th token. Logit: 13.77 Prob: 11.90% Token: | 512|
Top 2th token. Logit: 13.02 Prob:  5.63% Token: |
|
Top 3th token. Logit: 12.42 Prob:  3.07% Token: | 128|
Top 4th token. Logit: 11.50 Prob:  1.23% Token: | 4|
Top 5th token. Logit: 11.45 Prob:  1.17% Token: | 64|
Top 6th token. Logit: 11.45 Prob:  1.16% Token: | 320|
Top 7th token. Logit: 11.44 Prob:  1.15% Token: | 384|
Top 8th token. Logit: 11.40 Prob:  1.11% Token: | 192|
Top 9th token. Logit: 11.18 Prob:  0.89% Token: | 240|


Ranks of the answer tokens: [(' 512', 1)]

In [17]:
example_prompt = "16 32 64 128 256 512"
example_answer = " 1024"
utils.test_prompt(example_prompt, example_answer, modeltest, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', '16', ' 32', ' 64', ' 128', ' 256', ' 512']
Tokenized answer: [' 1024']


Performance on answer token:
Rank: 0        Logit: 14.54 Prob: 17.04% Token: | 1024|

Top 0th token. Logit: 14.54 Prob: 17.04% Token: | 1024|
Top 1th token. Logit: 13.37 Prob:  5.25% Token: | 1|
Top 2th token. Logit: 13.12 Prob:  4.10% Token: | 512|
Top 3th token. Logit: 12.94 Prob:  3.43% Token: | 768|
Top 4th token. Logit: 12.71 Prob:  2.74% Token: | 6|
Top 5th token. Logit: 12.70 Prob:  2.70% Token: | 8|
Top 6th token. Logit: 12.62 Prob:  2.50% Token: |
|
Top 7th token. Logit: 12.49 Prob:  2.18% Token: | 7|
Top 8th token. Logit: 12.21 Prob:  1.66% Token: | 5|
Top 9th token. Logit: 11.84 Prob:  1.14% Token: | MB|


Ranks of the answer tokens: [(' 1024', 0)]

In [ ]:
example_prompt = "16 32 64 128 256 512"
example_answer = " 1024"
utils.test_prompt(example_prompt, example_answer, modeltest, prepend_bos=True)

In [16]:
example_prompt = "10 12 14 16 18 20"
example_answer = " 22"
utils.test_prompt(example_prompt, example_answer, modeltest, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', '10', ' 12', ' 14', ' 16', ' 18', ' 20']
Tokenized answer: [' 22']


Performance on answer token:
Rank: 0        Logit: 15.67 Prob: 47.24% Token: | 22|

Top 0th token. Logit: 15.67 Prob: 47.24% Token: | 22|
Top 1th token. Logit: 14.36 Prob: 12.84% Token: | 21|
Top 2th token. Logit: 14.05 Prob:  9.35% Token: |
|
Top 3th token. Logit: 13.00 Prob:  3.30% Token: | 24|
Top 4th token. Logit: 12.82 Prob:  2.76% Token: | 2|
Top 5th token. Logit: 12.39 Prob:  1.79% Token: | Next|
Top 6th token. Logit: 12.03 Prob:  1.24% Token: | 23|
Top 7th token. Logit: 11.68 Prob:  0.88% Token: | 25|
Top 8th token. Logit: 11.64 Prob:  0.84% Token: | -|
Top 9th token. Logit: 11.56 Prob:  0.78% Token: | 20|


Ranks of the answer tokens: [(' 22', 0)]

# Generate dataset with multiple prompts

In [ ]:
class Dataset:
    def __init__(self, prompts, tokenizer, S1_is_first=False):
        self.prompts = prompts
        self.tokenizer = tokenizer
        self.N = len(prompts)
        self.max_len = max(
            [
                len(self.tokenizer(prompt["text"]).input_ids)
                for prompt in self.prompts
            ]
        )
        # all_ids = [prompt["TEMPLATE_IDX"] for prompt in self.ioi_prompts]
        all_ids = [0 for prompt in self.prompts] # only 1 template
        all_ids_ar = np.array(all_ids)
        self.groups = []
        for id in list(set(all_ids)):
            self.groups.append(np.where(all_ids_ar == id)[0])

        texts = [ prompt["text"] for prompt in self.prompts ]
        self.toks = torch.Tensor(self.tokenizer(texts, padding=True).input_ids).type(
            torch.int
        )
        self.io_tokenIDs = [
            self.tokenizer.encode(" " + prompt["corr"])[0] for prompt in self.prompts
        ]
        self.s_tokenIDs = [
            self.tokenizer.encode(" " + prompt["incorr"])[0] for prompt in self.prompts
        ]

        # word_idx: for every prompt, find the token index of each target token and "end"
        # word_idx is a tensor with an element for each prompt. The element is the targ token's ind at that prompt
        self.word_idx = {}
        for targ in [key for key in self.prompts[0].keys() if (key != 'text' and key != 'corr' and key != 'incorr')]:
            targ_lst = []
            for prompt in self.prompts:
                input_text = prompt["text"]
                tokens = model.tokenizer.tokenize(input_text)
                if S1_is_first and targ == "S1":  # only use this if first token doesn't have space Ġ in front
                    target_token = prompt[targ]
                else:
                    target_token = "Ġ" + prompt[targ]
                target_index = tokens.index(target_token)
                targ_lst.append(target_index)
            self.word_idx[targ] = torch.tensor(targ_lst)

        targ_lst = []
        for prompt in self.prompts:
            input_text = prompt["text"]
            tokens = self.tokenizer.tokenize(input_text)
            end_token_index = len(tokens) - 1
            targ_lst.append(end_token_index)
        self.word_idx["end"] = torch.tensor(targ_lst)

    def __len__(self):
        return self.N

Repalce io_tokens with correct answer (next, which is '5') and s_tokens with incorrect (current, which repeats)

In [ ]:
def generate_prompts_list(x ,y):
    prompts_list = []
    for i in range(x, y):
        prompt_dict = {
            'S1': str(i),
            'S2': str(i+2),
            'S3': str(i+4),
            'S4': str(i+6),
            'S5': str(i+8),
            'S6': str(i+10),
            'corr': str(i+12),
            'incorr': str(i+10),
            'text': f"{i} {i+2} {i+4} {i+6} {i+8} {i+10}"
        }
        prompts_list.append(prompt_dict)
    return prompts_list

prompts_list = generate_prompts_list(1, 11)
dataset = Dataset(prompts_list, model.tokenizer, S1_is_first=True)

In [ ]:
def generate_prompts_list_corr(x ,y):
    prompts_list = []
    for i in range(x, y):
        prompt_dict = {
            'S1': str(i),
            'S2': str(i+2),
            'S3': str(i+4),
            'S4': str(i+6),
            'S5': str(i+8),
            'S6': str(i+8),
            'corr': str(i+10),
            'incorr': str(i+12),
            'text': f"{i} {i+2} {i+4} {i+6} {i+8} {i+8}"
        }
        prompts_list.append(prompt_dict)
    return prompts_list

prompts_list_2 = generate_prompts_list_corr(1, 11)
dataset_2 = Dataset(prompts_list_2, model.tokenizer, S1_is_first=True)

Logit diff is correct - incorr token. Here, correct is S5, and incorr is S4.

Because of this, it's possible to have logit diffs HIGHER than the "full circuit" because the correct token will still be at first place, but the logit scores assigned will just be bigger (perhaps incorrect is scored even lower in the non-full circuit with a higher logit diff score)?

# Ablation Expm Functions

In [ ]:
from torch import Tensor

def logits_to_ave_logit_diff_2(logits: Float[Tensor, "batch seq d_vocab"], dataset: Dataset, per_prompt=False):
    '''
    Returns logit difference between the correct and incorrect answer.

    If per_prompt=True, return the array of differences rather than the average.
    '''

    # Only the final logits are relevant for the answer
    # Get the logits corresponding to the indirect object / subject tokens respectively
    io_logits: Float[Tensor, "batch"] = logits[range(logits.size(0)), dataset.word_idx["end"], dataset.io_tokenIDs]
    s_logits: Float[Tensor, "batch"] = logits[range(logits.size(0)), dataset.word_idx["end"], dataset.s_tokenIDs]
    # Find logit difference
    answer_logit_diff = io_logits - s_logits
    return answer_logit_diff if per_prompt else answer_logit_diff.mean()

We can also prevent redundant computation of the full circuit score by storing it and just passing it in to the function.

In [ ]:
ioi_logits_original, ioi_cache = model.run_with_cache(dataset.toks)
orig_score = logits_to_ave_logit_diff_2(ioi_logits_original, dataset)

In [ ]:
model.reset_hooks(including_permanent=True)  #must do this after running with mean ablation hook
ioi_logits_original, ioi_cache = model.run_with_cache(dataset.toks)
logits_to_ave_logit_diff_2(ioi_logits_original, dataset)

tensor(2.3903, device='cuda:0')

In [ ]:
def mean_ablate_by_lst(CIRCUIT, SEQ_POS_TO_KEEP, model, orig_score, print_output=True):
    model.reset_hooks(including_permanent=True)  #must do this after running with mean ablation hook

    # ioi_logits_original, ioi_cache = model.run_with_cache(dataset.toks)

    model_abl = ioi_circuit_extraction.add_mean_ablation_hook(model, means_dataset=dataset_2, circuit=CIRCUIT, seq_pos_to_keep=SEQ_POS_TO_KEEP)
    ioi_logits_minimal = model_abl(dataset.toks)

    # orig_score = logits_to_ave_logit_diff_2(ioi_logits_original, dataset)

    new_score = logits_to_ave_logit_diff_2(ioi_logits_minimal, dataset)
    if print_output:
        # print(f"Average logit difference (IOI dataset, using entire model): {orig_score:.4f}")
        # print(f"Average logit difference (IOI dataset, only using circuit): {new_score:.4f}")
        print(f"Average logit difference (circuit / full) %: {100 * new_score / orig_score:.4f}")
    # return new_score
    return 100 * new_score / orig_score

In [ ]:
lst = [(0, 0)]

CIRCUIT = {
    "number mover": lst,
    "number mover 5": lst,
    "number mover 4": lst,
    "number mover 3": lst,
    "number mover 2": lst,
    "number mover 1": lst,
}

SEQ_POS_TO_KEEP = {
    "number mover": "end",
    "number mover 5": "S5",
    "number mover 4": "S4",
    "number mover 3": "S3",
    "number mover 2": "S2",
    "number mover 1": "S1",
}

model.reset_hooks(including_permanent=True)  #must do this after running with mean ablation hook
model_abl = ioi_circuit_extraction.add_mean_ablation_hook(model, means_dataset=dataset_2, circuit=CIRCUIT, seq_pos_to_keep=SEQ_POS_TO_KEEP)
ioi_logits_minimal = model_abl(dataset.toks)
logits_to_ave_logit_diff_2(ioi_logits_minimal, dataset)

tensor(-0.4147, device='cuda:0')

# Ablate the model and compare with original

In [ ]:
lst = [(layer, head) for layer in range(24) for head in range(16)]

CIRCUIT = {
    "number mover": lst,
    "number mover 5": lst,
    "number mover 4": lst,
    "number mover 3": lst,
    "number mover 2": lst,
    "number mover 1": lst,
}

SEQ_POS_TO_KEEP = {
    "number mover": "end",
    "number mover 5": "S5",
    "number mover 4": "S4",
    "number mover 3": "S3",
    "number mover 2": "S2",
    "number mover 1": "S1",
}

mean_ablate_by_lst(CIRCUIT, SEQ_POS_TO_KEEP, model, orig_score, print_output=False).item()

100.0

In [ ]:
def circuit_from_headsList(headsList):
    CIRCUIT = {
    "number mover": headsList,
    "number mover 5": headsList,
    "number mover 4": headsList,
    "number mover 3": headsList,
    "number mover 2": headsList,
    "number mover 1": headsList,
    }

    SEQ_POS_TO_KEEP = {
        "number mover": "end",
        "number mover 5": "S5",
        "number mover 4": "S4",
        "number mover 3": "S3",
        "number mover 2": "S2",
        "number mover 1": "S1",
    }
    return CIRCUIT, SEQ_POS_TO_KEEP

In [ ]:
CIRCUIT, SEQ_POS_TO_KEEP = circuit_from_headsList([(0,0)])
mean_ablate_by_lst(CIRCUIT, SEQ_POS_TO_KEEP, model, orig_score, print_output=False).item()

-17.349857330322266

In [ ]:
# Start with full circuit
curr_circuit = [(layer, head) for layer in range(24) for head in range(16)]
threshold = 3  # This is T, a %. if performance is less than T%, allow its removal

for layer in range(23, -1, -1):  # go thru all heads in a layer first
    for head in range(16):
        # Copying the curr_circuit so we can iterate over one and modify the other
        copy_circuit = curr_circuit.copy()

        # Temporarily removing the current tuple from the copied circuit
        copy_circuit.remove((layer, head))

        CIRCUIT, SEQ_POS_TO_KEEP = circuit_from_headsList(copy_circuit)
        new_score = mean_ablate_by_lst(CIRCUIT, SEQ_POS_TO_KEEP, model, orig_score, print_output=False).item()

        print((layer,head), new_score)
        # If the result is less than the threshold, remove the tuple from the original list
        if (100 - new_score) < threshold:
            curr_circuit.remove((layer, head))

            print("Removed:", (layer, head))
            print(new_score)
            print("\n")

(23, 0) 100.06151580810547
Removed: (23, 0)
100.06151580810547


(23, 1) 100.24736785888672
Removed: (23, 1)
100.24736785888672


(23, 2) 100.31312561035156
Removed: (23, 2)
100.31312561035156


(23, 3) 100.19401550292969
Removed: (23, 3)
100.19401550292969


(23, 4) 100.26827239990234
Removed: (23, 4)
100.26827239990234


(23, 5) 99.635986328125
Removed: (23, 5)
99.635986328125


(23, 6) 99.55542755126953
Removed: (23, 6)
99.55542755126953


(23, 7) 99.43858337402344
Removed: (23, 7)
99.43858337402344


(23, 8) 97.73435974121094
Removed: (23, 8)
97.73435974121094


(23, 9) 97.71875
Removed: (23, 9)
97.71875


(23, 10) 97.73104095458984
Removed: (23, 10)
97.73104095458984


(23, 11) 97.63200378417969
Removed: (23, 11)
97.63200378417969


(23, 12) 97.68891143798828
Removed: (23, 12)
97.68891143798828


(23, 13) 97.6260986328125
Removed: (23, 13)
97.6260986328125


(23, 14) 97.65040588378906
Removed: (23, 14)
97.65040588378906


(23, 15) 97.73230743408203
Removed: (23, 15)
97.73230743408

In [ ]:
curr_circuit

[(0, 2),
 (0, 3),
 (0, 4),
 (0, 5),
 (0, 9),
 (0, 10),
 (0, 14),
 (1, 2),
 (1, 4),
 (1, 7),
 (1, 14),
 (2, 3),
 (2, 4),
 (2, 5),
 (2, 7),
 (2, 8),
 (2, 9),
 (2, 15),
 (3, 0),
 (3, 3),
 (3, 13),
 (3, 14),
 (3, 15),
 (4, 2),
 (4, 6),
 (4, 8),
 (4, 10),
 (4, 11),
 (5, 8),
 (6, 14),
 (6, 15),
 (7, 2),
 (7, 11),
 (7, 13),
 (8, 0),
 (9, 3),
 (9, 4),
 (9, 5),
 (9, 6),
 (9, 12),
 (9, 15),
 (10, 1),
 (10, 4),
 (10, 9),
 (10, 10),
 (10, 13),
 (10, 14),
 (11, 1),
 (11, 4),
 (11, 5),
 (11, 8),
 (12, 1),
 (12, 4),
 (12, 12),
 (12, 13),
 (12, 15),
 (13, 5),
 (13, 12),
 (13, 13),
 (14, 5),
 (14, 14),
 (15, 5),
 (15, 7),
 (15, 11),
 (15, 12),
 (15, 15),
 (16, 6),
 (16, 7),
 (16, 9),
 (16, 11),
 (16, 13),
 (16, 14),
 (17, 0),
 (17, 1),
 (17, 12),
 (18, 3),
 (18, 11),
 (18, 13),
 (19, 1),
 (19, 4),
 (20, 0),
 (20, 1),
 (20, 14),
 (21, 0),
 (21, 2),
 (21, 7)]

In [ ]:
len(curr_circuit)

86

384 total